# impute_missing

> Fill in a module description here

In [ ]:
#| default_exp impute_missing

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| export
from nbdev.showdoc import *
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from scipy.stats import truncnorm

In [ ]:
#| export
def impute_proteomics_data(df, conditions):
    def impute_detection_limit(condition_df, detection_limit):
        std_dev = detection_limit * 0.2

        def generate_positive_random_value(mean, std_dev):
            a, b = 0, np.inf
            return truncnorm.rvs(a=(a - mean) / std_dev, b=(b - mean) / std_dev, loc=mean, scale=std_dev)

        imputed_df = condition_df.applymap(lambda x: generate_positive_random_value(detection_limit, std_dev) if pd.isnull(x) else x)
        return imputed_df

    def compute_detection_limit(condition_df):
        smallest_values = []
        for column in condition_df.columns:
            column_data = condition_df[column]
            column_non_zero = column_data[column_data > 0]
            smallest_values.extend(column_non_zero.nsmallest(10).tolist())

        detection_limit = np.median(smallest_values)
        print('fill na with detection_limit:', detection_limit, np.log10(detection_limit), condition_df.columns)
        return detection_limit

    unique_conditions = set(conditions)
    imputed_dfs = []

    detection_limit_imputed_indices = []
    iterative_imputer_imputed_indices = []

    for condition in unique_conditions:
        condition_indices = [i for i, c in enumerate(conditions) if c == condition]
        condition_df = df.iloc[:, condition_indices]

        all_values_missing = condition_df.isnull().all(axis=1)
        missing_rows = condition_df[all_values_missing]
        non_missing_rows = condition_df[~all_values_missing]

        detection_limit = compute_detection_limit(condition_df)

        if not missing_rows.empty:
            imputed_missing_rows = impute_detection_limit(missing_rows, detection_limit)
            detection_limit_imputed_indices.extend(imputed_missing_rows.index.tolist())
        else:
            imputed_missing_rows = missing_rows

        imp_mean = IterativeImputer(random_state=0, imputation_order='roman')

        imputed_data = imp_mean.fit_transform(non_missing_rows)
        imputed_data = np.maximum(imputed_data, detection_limit)

        imputed_non_missing_rows = pd.DataFrame(imputed_data,
                                                index=non_missing_rows.index,
                                                columns=non_missing_rows.columns)

        combined_df = pd.concat([imputed_missing_rows, imputed_non_missing_rows]).sort_index()
        imputed_dfs.append(combined_df)

        # Get indices of rows imputed by IterativeImputer
        imputed_rows_mask = non_missing_rows.isnull().any(axis=1)
        iterative_imputer_imputed_indices.extend(non_missing_rows[imputed_rows_mask].index.tolist())

    imputed_df = pd.concat(imputed_dfs, axis=1).reindex(columns=df.columns)
    return imputed_df, detection_limit_imputed_indices, iterative_imputer_imputed_indices

In [ ]:
# Example usage
data = {
    "A1": [1,    2, 3, np.nan, 5,4,4],
    "A2": [2,    3, 4, np.nan, 6,4,np.nan],
    "A3": [1, np.nan, 3, np.nan, 5,np.nan,np.nan],
    "B1": [10, 11, 1, 13, np.nan,20,40],
    "B2": [11, 12, np.nan, 14, np.nan,25,np.nan],
    "B3": [10, 11, 1, 13, np.nan,np.nan, np.nan]
}
df = pd.DataFrame(data)
df

,A1,A2,A3,B1,B2,B3
0,1.0,2.0,1.0,10.0,11.0,10.0
1,2.0,3.0,NaN,11.0,12.0,11.0
2,3.0,4.0,3.0,1.0,NaN,1.0
3,NaN,NaN,NaN,13.0,14.0,13.0
4,5.0,6.0,5.0,NaN,NaN,NaN
5,4.0,4.0,NaN,20.0,25.0,NaN
6,4.0,NaN,NaN,40.0,NaN,NaN


In [ ]:
conditions = ["A", "A", "A", "B", "B", "B"]
imputed_df = impute_proteomics_data(df, conditions)
imputed_df

detection_limit 11.5
detection_limit 3.5


,A1,A2,A3,B1,B2,B3
0,3.50000,3.500000,3.500000,11.500000,11.500000,11.500000
1,3.50000,3.500000,3.500000,11.500000,12.000000,11.500000
2,3.50000,4.000000,3.500000,11.500000,11.500000,11.500000
3,2.94932,4.251355,2.497197,13.000000,14.000000,13.000000
4,5.00000,6.000000,5.000000,11.356103,12.932057,11.023701
5,4.00000,4.000000,3.500000,20.000000,25.000000,21.999991
6,4.00000,4.297517,3.648759,40.000000,52.111114,45.555528


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()